**COMPILADORES - AULA 04**

**Prof. Luciano Silva**

**OBJETIVOS DA AULA:**



*   Revisar o processo de análise sintática
*   Implementar um analisador sintático para a linguagem TINY-C



In [1]:
!pip install rply

**REVISÃO DO PROCESSO DE ANÁLISE SINTÁTICA**

Na nossa última aula, implementamos um analisador sintático completo para o comando de atribuição com expressões ariméticas envolvendo números inteiros sem sinal:

\<atrib\>::= ID "=" \<expression\>

\<expression\> ::= NUMBER

       | \<expression\> "+" \<expression\>
 
       | \<expression\> "-" \<expression\>
 
       | \<expression\> "*" \<expression\>
 
       | \<expression\> "/" \<expression\>
 
       | "(" <expression> ")"

O primeiro passo foi implementar um analisador léxico para esta gramática, mostrado abaixo:


In [2]:
from rply import LexerGenerator

lg = LexerGenerator()

lg.add('ID', r'[a-zA-Z][a-zA-Z0-9]*')
lg.add('EQUALS', r'=')
lg.add('NUMBER', r'\d+')
lg.add('PLUS', r'\+')
lg.add('MINUS', r'-')
lg.add('MUL', r'\*')
lg.add('DIV', r'/')
lg.add('OPEN_PARENS', r'\(')
lg.add('CLOSE_PARENS', r'\)')

lg.ignore('\s+')

lexer = lg.build()

O segundo passo foi implementar as classes em Python para representar os nós da árvore sintática gerada pelo analisador sintático:

In [3]:
from rply.token import BaseBox

class Number(BaseBox):
    def __init__(self, value):
        self.value = value

    def eval(self):
        return self.value

class BinaryOp(BaseBox):
    def __init__(self, left, right):
        self.left = left
        self.right = right

class Add(BinaryOp):
    def eval(self):
        return self.left.eval() + self.right.eval()

class Sub(BinaryOp):
    def eval(self):
        return self.left.eval() - self.right.eval()

class Mul(BinaryOp):
    def eval(self):
        return self.left.eval() * self.right.eval()

class Div(BinaryOp):
    def eval(self):
        return self.left.eval() / self.right.eval()

class Attrib(BaseBox):
    def __init__(self, id, expression):
        self.id = id
        self.expression = expression

Finalmente, foi implementado o analisado sintático para o comando de atribuição:

In [4]:
from rply import ParserGenerator

pg = ParserGenerator(
    # A list of all token names, accepted by the lexer.
    ['NUMBER', 'OPEN_PARENS', 'CLOSE_PARENS',
     'PLUS', 'MINUS', 'MUL', 'DIV','ID','EQUALS'
    ],
    # A list of precedence rules with ascending precedence, to
    # disambiguate ambiguous production rules.
    precedence=[
        ('left', ['PLUS', 'MINUS']),
        ('left', ['MUL', 'DIV'])    
    ]
)

# regra <atrib>::= ID "=" <expression>

@pg.production('atrib : ID EQUALS expression')
def attrib(p):
  return Attrib(p[0].getstr(),p[2])

@pg.production('expression : NUMBER')
def expression_number(p):
    # p is a list of the pieces matched by the right hand side of the
    # rule
    return Number(int(p[0].getstr()))

@pg.production('expression : OPEN_PARENS expression CLOSE_PARENS')
def expression_parens(p):
    return p[1]

@pg.production('expression : expression PLUS expression')
@pg.production('expression : expression MINUS expression')
@pg.production('expression : expression MUL expression')
@pg.production('expression : expression DIV expression')
def expression_binop(p):
    left = p[0]
    right = p[2]
    if p[1].gettokentype() == 'PLUS':
        return Add(left, right)
    elif p[1].gettokentype() == 'MINUS':
        return Sub(left, right)
    elif p[1].gettokentype() == 'MUL':
        return Mul(left, right)
    elif p[1].gettokentype() == 'DIV':
        return Div(left, right)
    else:
        raise AssertionError('Oops, this should not be possible!')

parser = pg.build()

Realizamos um teste com um comando de atribuição:

In [5]:
arvore=parser.parse(lexer.lex('x=1+2*3'))
print(arvore)
print(arvore.id)
print(arvore.expression.eval())

x
7


**EXERCÍCIO**

*Implementar e testar um analisador sintático para a gramática da linguagem TINY-C. Todos os nós da árvore sintática devem implementar o método print que, quando invocado, deve mostrar todo o conteúdo armazenado nos seus atributos.*




program ::= function (function)*

function ::= type_specifier id ‘(‘ param_decl_list ‘)’ compound_stmt

type_specifier ::= int | char

param_decl_list ::= parameter_decl (‘,’ parameter_decl )*

param_decl ::= type_specifier id

compound_stmt ::= ‘{‘ (var_decl stmt)? ‘}’

var_decl ::= type_specifier var_decl_list ‘;’

var_decl_list ::= variable_id ( ‘,’ variable_id)*

variable_id ::= id ( ‘=’ expr )? | id '[' num ']'

stmt ::= compound_stmt | cond_stmt | while_stmt | break ‘;’ | continue ‘;’ | return expr ‘;’ | readint ‘(‘ id ‘)’ ‘;’ | writeint ‘(‘ expr ‘)’ ‘;’

cond_stmt ::= if ‘(‘ expr ‘)’ stmt (else stmt)?

while_stmt ::= while ‘(‘ expr ‘)’ stmt

expr ::= id ‘=’ expr | condition

condition ::= disjunction | disjunction ‘?’ expr ‘:’ condition

disjunction ::= conjunction | disjunction ‘||’ conjunction

conjunction ::= comparison | conjunction ‘&&’ comparison

comparison ::= relation | relation ‘==’ relation

relation ::= sum | sum (‘<’ | ‘>’) sum

sum ::= sum ‘+’ term | sum ‘-’ term | term

term ::= term ‘*’ factor | term ‘/’ factor | term ‘%’ factor | factor

factor ::= ‘!’ factor | ‘-’ factor | primary

primary ::= num | charconst | id | ‘(‘ expr ‘)’ 

A implementação do analisador léxico está disponível abaixo:

In [ ]:
from rply import LexerGenerator

lg = LexerGenerator()

lg.add('INT', r'int')
lg.add('CHAR', r'char')
lg.add('BREAK', r'break')
lg.add('CONTINUE', r'continue')
lg.add('RETURN', r'return')
lg.add('READINT', r'readint')
lg.add('WRITEINT', r'writeint')
lg.add('IF', r'if')
lg.add('WHILE', r'while')
lg.add('ID', r'[a-zA-Z][a-zA-Z0-9]*')
lg.add('OPEN_PAR', r'\(')
lg.add('CLOSE_PAR', r'\)')
lg.add('OPEN_COL', r'\[')
lg.add('CLOSE_COL', r'\]')
lg.add('VIRG', r'\,')
lg.add('OPEN_CH', r'\{')
lg.add('CLOSE_CH', r'\}')
lg.add('PVIRG', r'\;')
lg.add('COMPEQUALS', r'==')
lg.add('COMPMAIOR', r'\>')
lg.add('COMPMENOR', r'\<')
lg.add('EQUALS', r'=')
lg.add('INTERROG', r'\?')
lg.add('DOISP', r'\:')
lg.add('DISJ', r'\|\|')
lg.add('CONJ', r'&&')
lg.add('NOT', r'\!')
lg.add('NUMBER', r'\d+')
lg.add('CHARCONST', r'\'\S\'')
lg.add('PLUS', r'\+')
lg.add('MINUS', r'-')
lg.add('MUL', r'\*')
lg.add('DIV', r'/')
lg.add('MOD', r'\%')

lg.ignore('\s+')

lexer = lg.build()

In [ ]:
#implemente suas classes dos nós da árvore sintática aqui
class Program(BaseBox):
    def __init__(self, function, functions):
        self.function = function
        self.functions = functions
      
    def print(self):
        self.function.print()
        for f in self.functions:
          f.print()


class Function(BaseBox):
    def __init__(self, id, dtype, param, body):
        self.id = id
        self.dtype = dtype
        self.param = param
        self.body = body
    
    def print(self):
      self.id.print()
      self.dtype.print()
      self.param.print()
      self.body.print()

#param_decl_list ::= parameter_decl (‘,’ parameter_decl )*
class Param_declaration(BaseBox):
      def __init__(self, parameter_decl, parameter_decl_list):
        self.parameter_decl = parameter_decl
        self.parameter_decl_list = parameter_decl_list
      
      def print(self):
        self.parameter_decl.print()
        for p in parameter_decl_list:
          p.print()



class Condition(BaseBox):
    def __init__()
      pass

class 

class Mod(BinaryOp):
    def eval(self):
      return self.left.eval()%self.right.eval()

In [ ]:
#implemente seu analisador sintático aqui
from rply import ParserGenerator

pg = ParserGenerator(
    # A list of all token names, accepted by the lexer.
    ['NUMBER', 'OPEN_PARENS', 'CLOSE_PARENS',
     'PLUS', 'MINUS', 'MUL', 'DIV','ID','EQUALS'
     'MOD'
    ],
    # A list of precedence rules with ascending precedence, to
    # disambiguate ambiguous production rules.
    precedence=[
        ('left', ['PLUS', 'MINUS']),
        ('left', ['MUL', 'DIV', 'MOD'])    
    ]
)

# regra <atrib>::= ID "=" <expression>

@pg.production('atrib : ID EQUALS expression')
def attrib(p):
  return Attrib(p[0].getstr(),p[2])

@pg.production('expression : NUMBER')
def expression_number(p):
    # p is a list of the pieces matched by the right hand side of the
    # rule
    return Number(int(p[0].getstr()))

@pg.production('expression : OPEN_PARENS expression CLOSE_PARENS')
def expression_parens(p):
    return p[1]

@pg.production('expression : expression PLUS expression')
@pg.production('expression : expression MINUS expression')
@pg.production('expression : expression MUL expression')
@pg.production('expression : expression DIV expression')
@pg.production('expression : expression MOD expression')

def expression_binop(p):
    left = p[0]
    right = p[2]
    if p[1].gettokentype() == 'PLUS':
        return Add(left, right)
    elif p[1].gettokentype() == 'MINUS':
        return Sub(left, right)
    elif p[1].gettokentype() == 'MUL':
        return Mul(left, right)
    elif p[1].gettokentype() == 'DIV':
        return Div(left, right)
    elif p[1]
    else:
        raise AssertionError('Oops, this should not be possible!')

parser = pg.build()

In [ ]:
#teste seu analisador com um pequeno programa em TINY-C.

**ATIVIDADE EAD**

Finalizar a implementação do analisador sintático para TINY-C.

